In [171]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Maternal Mortality Ratio

In [172]:
# Data source: https://www.gapminder.org/data/
# See section: Data/Health/Maternal health/Maternal Mortality
source = pd.read_csv('maternal_mortality_ratio_per_100000_live_births.csv')
source = source.set_index('country').replace(0, np.nan)

non_missing = source.shape[0] - source.isnull().sum()
df = source[list(non_missing[non_missing >=180].index)].dropna().unstack().reset_index()
df.columns = ['year', 'country', 'mm_ratio']

In [173]:
df.sample(5)

,year,country,mm_ratio
881,2005,Togo,510.0
202,1990,Brunei,26.0
227,1990,Ecuador,160.0
1193,2013,Netherlands,6.0
77,1980,Israel,8.6


## Gapminer Region Groups

In [174]:
#data source: https://www.gapminder.org/data/geo/
regions = pd.read_csv('list-of-countries-etc.csv')

# subsetting regional variables
cols_to_keep = ['geo', 'name', 'eight_regions']
regions = regions[cols_to_keep]
    
#merging region info with maternal mortality data
df = pd.merge(df, regions, 
              how = 'left', 
              left_on = 'country', 
              right_on = 'name')

df = df.drop('name', axis=1)
df['eight_regions'] = df['eight_regions'].astype('category')
df = df.rename(columns={'eight_regions': 'region'})

# I did this replacement for better readability in Tableau,
# comment to keep original alphabetical sorting
df = df.replace({'region' : { 'africa_north' : 'North Africa', 
                             'africa_sub_saharan': 'Sub-Saharan Africa', 
                             'america_north': 'North America',
                             'america_south': 'South America', 
                             'asia_west': 'West Asia', 
                             'east_asia_pacific': 'East Asia and Pacific', 
                             'europe_east': 'East Europe', 
                             'europe_west': 'West Europe'
                            }
                })

In [175]:
df.sample(5)

,year,country,mm_ratio,geo,region
793,2005,Indonesia,250.0,idn,East Asia and Pacific
714,2000,Vanuatu,120.0,vut,East Asia and Pacific
180,1990,Afghanistan,1200.0,afg,West Asia
354,1990,Vanuatu,170.0,vut,East Asia and Pacific
1132,2013,Estonia,11.0,est,East Europe


## World Bank Income Classification

In [176]:
# Data source: https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups

# No data for 1980

wb_income = pd.read_csv('WB_income.csv')
wb_income = wb_income.replace("..", np.nan)
wb_income = wb_income.drop(['country'], axis=1)

#converting geocodes to lowercase for merging
wb_income['code'] = wb_income['code'].str.lower()

wb_income = wb_income.set_index('code').unstack().reset_index()
wb_income.columns = ['year', 'code', 'income']

df = pd.merge(df, wb_income, 
              how = 'left', 
              left_on = ['year', 'geo'], 
              right_on = ['year', 'code'], 
              suffixes=('', '_income'))
df = df.drop(['code'], axis=1)

df['income_rank'] = df['income']
df = df.replace({'income_rank' : { 'H' : 4, 'UM' : 3, 'LM' : 2, 'L': 1 }})
df = df.replace({'income' : { 'H' : 'High', 'UM' : 'Upper middle', 'LM' : 'Lower middle', 'L': 'Low' }})

In [177]:
df.sample(5)

,year,country,mm_ratio,geo,region,income,income_rank
1095,2013,Belize,45.0,blz,North America,Upper middle,3.0
811,2005,Libya,17.0,lby,North Africa,Upper middle,3.0
805,2005,Kyrgyz Republic,92.0,kgz,West Asia,Low,1.0
731,2005,Bangladesh,260.0,bgd,West Asia,Low,1.0
1233,2013,Swaziland,310.0,swz,Sub-Saharan Africa,Lower middle,2.0


## Attended Births Data

In [178]:
# Data source: https://www.gapminder.org/data/
# See section: Data/Health/Maternal health/Births attended by skilled health staff (% of total)

# No data before 1984

attended_births = pd.read_csv('births_attended_by_skilled_health_staff_percent_of_total.csv')
attended_births = attended_births.replace(0, np.nan)
attended_births = attended_births.set_index('country').unstack().reset_index()
attended_births.columns = ['year', 'country', 'attened_births_perc']

df = pd.merge(df, attended_births, 
              how = 'left', 
              on=['year', 'country'],
              suffixes=('', '_ab'))

In [179]:
df.sample(5)

,year,country,mm_ratio,geo,region,income,income_rank,attened_births_perc
929,2010,Cape Verde,58.0,cpv,Sub-Saharan Africa,Lower middle,2.0,98.5
248,1990,Haiti,670.0,hti,North America,Low,1.0,NaN
47,1980,Ecuador,288.0,ecu,South America,NaN,NaN,NaN
1155,2013,Iraq,67.0,irq,West Asia,Upper middle,3.0,NaN
835,2005,Nicaragua,120.0,nic,North America,Lower middle,2.0,NaN


## Total Health Spending Per Person, USD

In [180]:
# Data source: https://www.gapminder.org/data/
# See section: Data/Health/Health Economics/Total health spending / person, US$ 

# No data before 1995

total_health_spending = pd.read_csv('total_health_spending_per_person_us.csv')
total_health_spending = total_health_spending.replace(0, np.nan)
total_health_spending = total_health_spending.set_index('country').unstack().reset_index()
total_health_spending.columns = ['year', 'country', 'total_health_spending_pp']

df = pd.merge(df, total_health_spending, 
              how = 'left', 
              on=['year', 'country'],
              suffixes=('', '_thspp'))

In [181]:
df.sample(5)

,year,country,mm_ratio,geo,region,income,income_rank,attened_births_perc,total_health_spending_pp
145,1980,South Africa,208.0,zaf,Sub-Saharan Africa,NaN,NaN,NaN,NaN
53,1980,Ethiopia,1060.0,eth,Sub-Saharan Africa,NaN,NaN,NaN,NaN
1226,2013,South Korea,27.0,kor,East Asia and Pacific,High,4.0,NaN,NaN
172,1980,Uruguay,54.9,ury,South America,NaN,NaN,NaN,NaN
417,1995,Gabon,340.0,gab,Sub-Saharan Africa,Upper middle,3.0,NaN,136.0


## Goverment Share Of Total Health Spending Per Person, \%

In [182]:
# Data source: https://www.gapminder.org/data/
# See section: Data/Health/Health Economics/Govt. share of health spending (%)

# No data before 1995

gov_share_health_spending = pd.read_csv('government_share_of_total_health_spending_percent.csv')
gov_share_health_spending = gov_share_health_spending.replace(0, np.nan)
gov_share_health_spending = gov_share_health_spending.set_index('country').unstack().reset_index()
gov_share_health_spending.columns = ['year', 'country', 'gov_share_thspp']

df = pd.merge(df, gov_share_health_spending, 
              how = 'left', 
              on=['year', 'country'],
              suffixes=('', '_gsthspp'))


In [183]:
df.sample(5)

,year,country,mm_ratio,geo,region,income,income_rank,attened_births_perc,total_health_spending_pp,gov_share_thspp
163,1980,Trinidad and Tobago,68.1,tto,North America,NaN,NaN,NaN,NaN,NaN
1057,2010,Tajikistan,48.0,tjk,West Asia,Low,1.0,87.7,49.1,26.7
943,2010,Czech Republic,5.0,cze,East Europe,High,4.0,99.7,1480.0,83.7
316,1990,Saudi Arabia,41.0,sau,West Asia,Upper middle,3.0,88.0,NaN,NaN
248,1990,Haiti,670.0,hti,North America,Low,1.0,NaN,NaN,NaN


## Total Fertility (Number Of Children Per Woman

In [184]:
# Data source: https://www.gapminder.org/data/
# See section: Data/Health/Newborn & Infants/Babies per woman

total_fert = pd.read_csv('children_per_woman_total_fertility.csv')
total_fert = total_fert.replace(0, np.nan)
total_fert = total_fert.set_index('country').unstack().reset_index()
total_fert.columns = ['year', 'country', 'total_fertility']

df = pd.merge(df, total_fert, 
              how = 'left', 
              on=['year', 'country'],
              suffixes=('', '_tf'))

In [185]:
df.sample(5)

,year,country,mm_ratio,geo,region,income,income_rank,attened_births_perc,total_health_spending_pp,gov_share_thspp,total_fertility
805,2005,Kyrgyz Republic,92.0,kgz,West Asia,Low,1.0,97.9,28.4,40.9,2.58
71,1980,Iceland,11.4,isl,West Europe,NaN,NaN,NaN,NaN,NaN,2.31
1211,2013,Romania,33.0,rou,East Europe,Upper middle,3.0,98.5,NaN,NaN,1.49
893,2005,Uzbekistan,44.0,uzb,West Asia,Low,1.0,100.0,29.0,43.3,2.46
479,1995,Norway,4.0,nor,West Europe,High,4.0,99.2,3120.0,80.3,1.87


## Median Age

In [186]:
# Data source: https://www.gapminder.org/data/
# See section: Data/Population/Median Age

median_age = pd.read_csv('median_age_years.csv')
median_age = median_age.replace(0, np.nan)
median_age = median_age.set_index('country').unstack().reset_index()
median_age.columns = ['year', 'country', 'median_age']

df = pd.merge(df, median_age, 
              how = 'left', 
              on=['year', 'country'],
              suffixes=('', '_ma'))

In [187]:
df.sample(5)

,year,country,mm_ratio,geo,region,income,income_rank,attened_births_perc,total_health_spending_pp,gov_share_thspp,total_fertility,median_age
733,2005,Belarus,21.0,blr,East Europe,Lower middle,2.0,99.9,212.0,72.9,1.32,38.1
453,1995,Luxembourg,11.0,lux,West Europe,High,4.0,NaN,2820.0,92.4,1.71,36.7
197,1990,Bhutan,900.0,btn,West Asia,Low,1.0,NaN,NaN,NaN,5.64,17.9
39,1980,Cote d'Ivoire,590.0,civ,Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN,7.60,17.4
902,2010,Algeria,92.0,dza,North Africa,Upper middle,3.0,NaN,178.0,77.9,2.89,26.0


In [194]:
# dropping geo codes for Tableau
df = df.drop('geo', axis=1)

In [169]:
df.to_csv('mm_merged_data_tableau.csv')